In [1]:
import flodym as fd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from flodym import StockArray

In [2]:
PV = np.array([110,131,152,173,194,215,234,253,272,291,310,328,346,364,382,400,416,432,448,464,480])

Batt = np.array([27.50,37.17,48.25,60.75,74.67,90.00,104.10,119.20,135.30,152.40,170.50,190.24,211.06,232.96,255.94,280.00,303.68,328.32,353.92,380.48,408.00
])

In [ ]:
# time_pv = fd.Dimension(name="Time", letter="t", items=list(range(2025, 2046)))
# dims_pv = fd.DimensionSet(dim_list=[time_pv])

# time_batt = fd.Dimension(name="Time", letter="t", items=list(range(2025, 2046)))
# dims_batt = fd.DimensionSet(dim_list=[time_batt])

# PV_stock_GWp = StockArray(dims=dims_pv)
# PV_stock_GWp.values[...] = PV

# Batt_stock_GWh = StockArray(dims=dims_batt)
# Batt_stock_GWh.values[...] = Batt

# lifetime_pv = fd.lifetime_models.LogNormalLifetime(dims=dims_pv, mean=20, std=4)
# lifetime_batt = fd.lifetime_models.LogNormalLifetime(dims=dims_batt, mean=10, std=2)

# dsm_pv = fd.StockDrivenDSM(
#     dims=dims_pv, 
#     lifetime_model=lifetime_pv,
#     stock=PV_stock_GWp
# )
# dsm_batt = fd.StockDrivenDSM(
#     dims=dims_batt, 
#     lifetime_model=lifetime_batt,
#     stock=Batt_stock_GWh
# )

# dsm_pv.compute()
# dsm_batt.compute()

# print("\n PV RESULTS")

# print("\n--- Stock (should match input) ---")
# print(dsm_pv.stock.to_df())

# print("\n--- Inflow (new installations needed) ---")
# print(dsm_pv.inflow.to_df())

# print("\n--- Outflow (end-of-life waste) ---")
# print(dsm_pv.outflow.to_df())

# print("\n BATTERY RESULTS")
# print("\n--- Stock (should match input) ---")
# print(dsm_batt.stock.to_df())

# print("\n--- Inflow (new installations needed) ---")
# print(dsm_batt.inflow.to_df())

# print("\n--- Outflow (end-of-life waste) ---")
# print(dsm_batt.outflow.to_df())


# print("VERIFICATION")

# # Check: Stock balance should hold: Stock[t] = Stock[t-1] + Inflow[t] - Outflow[t]
# pv_balance_check = (
#     dsm_pv.stock.values[1:] - dsm_pv.stock.values[:-1] -  dsm_pv.inflow.values[1:] + dsm_pv.outflow.values[1:])
# print(f"\nPV Stock balance error (should be ~0): {np.abs(pv_balance_check).max():.6f}")

# batt_balance_check = (
#     dsm_batt.stock.values[1:] - dsm_batt.stock.values[:-1] - 
#     dsm_batt.inflow.values[1:] + dsm_batt.outflow.values[1:]
# )
# print(f"Battery Stock balance error (should be ~0): {np.abs(batt_balance_check).max():.6f}")

# # Summary statistics
# print(f"\n--- PV Summary ---")
# print(f"Total inflow (2025-2045): {dsm_pv.inflow.values.sum():.1f} GWp")
# print(f"Total outflow (2025-2045): {dsm_pv.outflow.values.sum():.1f} GWp")
# print(f"Final stock (2045): {dsm_pv.stock.values[-1]:.1f} GWp")

# print(f"\n--- Battery Summary ---")
# print(f"Total inflow (2025-2045): {dsm_batt.inflow.values.sum():.1f} GWh")
# print(f"Total outflow (2025-2045): {dsm_batt.outflow.values.sum():.1f} GWh")
# print(f"Final stock (2045): {dsm_batt.stock.values[-1]:.1f} GWh")

In [3]:
# COMBINED MODEL: DOES NOT WORK####
time = fd.Dimension(name="Time", letter="t", items=list(range(2025, 2046)))
product = fd.Dimension(name = "Product", letter = "p", items=["PV", "Battery"])
dims = fd.DimensionSet(dim_list=[time, product])

Stock_total =StockArray(dims=dims)
Stock_total["PV"].values[...] = PV
Stock_total["Battery"].values[...] = Batt
print(Stock_total.to_df())

lifetime_mean = fd.FlodymArray(dims=dims["p",])
lifetime_mean["PV"] = 20.
lifetime_mean["Battery"] = 10.
    
lifetime_std = fd.FlodymArray(dims=dims["p",])
lifetime_std["PV"] = 4.
lifetime_std["Battery"] = 2.
    
lifetime = fd.lifetime_models.LogNormalLifetime(dims=dims, mean=lifetime_mean, std=lifetime_std)

dsm = fd.StockDrivenDSM(
    dims=dims, 
    lifetime_model=lifetime,
    stock=Stock_total
)
dsm.compute()

print("\n--- Stock (should match input) ---")
print(dsm.stock.to_df())

print("\n--- Inflow (new installations needed) ---")
print(dsm.inflow.to_df())

print("\n--- Outflow (end-of-life waste) ---")
print(dsm.outflow.to_df())

# Summary statistics
print(f"\n--- Summary ---")
print(f"Total inflow (2025-2045): {dsm.inflow.values.sum():.1f}")
print(f"Total outflow (2025-2045): {dsm.outflow.values.sum():.1f}")
print(f"Final stock (2045): {dsm.stock.values[-1]:.1f}")

               value
Time Product        
2025 PV       110.00
     Battery   27.50
2026 PV       131.00
     Battery   37.17
2027 PV       152.00
     Battery   48.25
2028 PV       173.00
     Battery   60.75
2029 PV       194.00
     Battery   74.67
2030 PV       215.00
     Battery   90.00
2031 PV       234.00
     Battery  104.10
2032 PV       253.00
     Battery  119.20
2033 PV       272.00
     Battery  135.30
2034 PV       291.00
     Battery  152.40
2035 PV       310.00
     Battery  170.50
2036 PV       328.00
     Battery  190.24
2037 PV       346.00
     Battery  211.06
2038 PV       364.00
     Battery  232.96
2039 PV       382.00
     Battery  255.94
2040 PV       400.00
     Battery  280.00
2041 PV       416.00
     Battery  303.68
2042 PV       432.00
     Battery  328.32
2043 PV       448.00
     Battery  353.92
2044 PV       464.00
     Battery  380.48
2045 PV       480.00
     Battery  408.00

--- Stock (should match input) ---
               value
Time Product       

TypeError: unsupported format string passed to numpy.ndarray.__format__